In [40]:
import requests
import lxml
from bs4 import BeautifulSoup
import pandas as pd
import plotly.express as px
import numpy as np

In [41]:
# user agent through which requests are sent
header = {
    'User-agent': 'Mozilla/5.0 (Linux; Android 11; SAMSUNG SM-G973U) AppleWebKit/537.36 (KHTML, like Gecko) SamsungBrowser/14.2 Chrome/87.0.4280.141 Mobile Safari/537.36'
}

In [42]:
page_no = []   # object to store URL criteria

n = 9   # insert number of pages

for x in range(1,n):  # page numbers 37
    categories = x   # multiply by number of items on page
    page_no.append(categories)

In [43]:
cos_dresses = []

for x in page_no:
    r = requests.get(f'https://www.cos.com/en_gbp/women/dresses.html?page={x}', headers=header)  # loop through pages
    soup = BeautifulSoup(r.content, 'lxml')
    
    products = soup.find_all('div', class_='column')  # Select the main product divs

    for product in products:
        # Check for the name element
        name_element = product.find('h2', class_='a-heading-2 product-title')
        name = name_element.text.strip() if name_element else 'Name Not Available'
        
        # Check for the price element
        price_element = product.find('div', class_='m-product-price')
        price = price_element.text.strip() if price_element else 'Price Not Available'
    
        # Collect product details
        product_details = {
            'name': name,
            'price': price
        }
    
        cos_dresses.append(product_details)


In [44]:
# write to dataframe
df = pd.DataFrame(cos_dresses)
print(df.head(15))  # print top 15

                                     name                price
0        GEOMETRIC-PRINT MAXI SHIRT DRESS                 £115
1   GATHERED SILK-BLEND VELVET MIDI DRESS                 £135
2            HERRINGBONE MAXI SHIRT DRESS                 £135
3             BRUSHED-CASHMERE MAXI DRESS                 £300
4                RUFFLED SWING MINI DRESS                  £85
5                  MERINO WOOL MIDI DRESS                  £95
6                        WOOL SHIRT DRESS                 £125
7                      Name Not Available  Price Not Available
8         SCARF-DETAIL PRINTED MIDI DRESS                 £115
9           BIB-FRONT CORDUROY MINI DRESS                  £85
10               CORSET-DETAIL MAXI DRESS                  £95
11           ASYMMETRIC MERINO WOOL DRESS                 £115
12            SATIN RACER-NECK MINI DRESS                 £110
13           MERINO WOOL TURTLENECK DRESS                  £95
14              DRAPED BALLOON MIDI DRESS              

In [47]:
# Filter out rows where the price is 'Price Not Available'
df = df[df['price'] != 'Price Not Available']

df['price'] = df['price'].replace('£','',regex=True)
df['price'] = df['price'].replace('From','',regex=True)
df['price'] = df['price'].replace(',','',regex=True)
df['price'] = pd.to_numeric(df['price'])
# Optionally, reset the index after removing other rows
df = df.reset_index(drop=True)

In [48]:
df

,name,price
0,GEOMETRIC-PRINT MAXI SHIRT DRESS,115
1,GATHERED SILK-BLEND VELVET MIDI DRESS,135
2,HERRINGBONE MAXI SHIRT DRESS,135
3,BRUSHED-CASHMERE MAXI DRESS,300
4,RUFFLED SWING MINI DRESS,85
...,...,...
518,DEEP V-NECK DRESS,95
519,TIERED DRESS,69
520,STRIPED WRAP DRESS,79
521,OVERSIZED T-SHIRT DRESS,65


In [49]:
df.to_excel('cos_dresses.xlsx', index=False) 

In [53]:
print(df['price'].mode())

0    95
Name: price, dtype: int64
